# Section 2A

## Import and load data
I'm using sklearn to create the decision tree itself, and matplotlib/graphviz to plot the result

In [1]:
from sklearn import tree
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
import graphviz

Create list of attributes that correspond to the columns in the data. To convert these codes to an actual attribute, look in the names file

In [2]:
ATTRIBUTES = pd.Index([
    "AAGE",
    "ACLSWKR",
    "ADTIND",
    "ADTOCC",
    # "AGI",
    "AHGA",
    "AHRSPAY",
    "AHSCOL",
    "AMARITL",
    "AMJIND",
    "AMJOCC",
    "ARACE",
    "AREORGN",
    "ASEX",
    "AUNMEM",
    "AUNTYPE",
    "AWKSTAT",
    "CAPGAIN",
    "CAPLOSS",
    "DIVVAL",
    # "FEDTAX",
    "FILESTAT",
    "GRINREG",
    "GRINST",
    "HHDFMX",
    "HHDREL",
    "MARSUPWT", #This is sample weight, will be dropped later
    "MIGMTR1",
    "MIGMTR3",
    "MIGMTR4",
    "MIGSAME",
    "MIGSUN",
    "NOEMP",
    "PARENT",
    # "PEARNVAL",
    "PEFNTVTY",
    "PEMNTVTY",
    "PENATVTY",
    "PRCITSHP",
    # "PTOTVAL",
    "SEOTR",
    # "TAXINC",
    "VETQVA",
    "VETYN",
    "WKSWORK",
    "YEAR",
    "RESULT"
])

Load data with pandas.   
Take the last column (RESULT) and put it in it's own vector.  
Drop the columns for result and instance weight so our classifier does not attempt to use those in the decision tree.

In [3]:
DATA = pd.read_csv('census-income.data', header=None, names=ATTRIBUTES)
RESULTS = DATA[:]['RESULT']

RESULTS = RESULTS.apply(lambda x: 0 if '-' in x else 1)
CLASS_NAMES=['-50000', '50000+']

DATA = DATA.drop(columns=["MARSUPWT","RESULT"])
ATTRIBUTE_COLUMNS = DATA.columns

## Format Data

Sklearn only produces binary trees, and we have columns that have multiple choices and are not continous.  
We use OneHotEncoder or pd.get_dummies to change those columns to multiple columns with only two choices, 1 or 0.   
  
Note: This cell can take a while. It took around 10 seconds on my fairly new laptop.

In [4]:
CLEANED_DATA = DATA.copy()
ohe = OneHotEncoder()
# These attributes are already integers, so get_dummies will ignore them. We check for them separately
numerical_non_continuous = {'ADTIND', 'ADTOCC','SEOTR','VETYN'}

for t, col in zip(DATA.dtypes, DATA.columns):
    if t == 'object':
        new_cols = pd.get_dummies(DATA[[col]])
        CLEANED_DATA = pd.concat([CLEANED_DATA, new_cols], axis=1)
        CLEANED_DATA = CLEANED_DATA.drop(columns=[col])
    if col in numerical_non_continuous:
        new_cols = pd.DataFrame(ohe.fit_transform(DATA[[col]]).toarray())
        new_cols.columns = ohe.get_feature_names_out()     
        CLEANED_DATA = pd.concat([CLEANED_DATA, new_cols], axis=1)
        CLEANED_DATA = CLEANED_DATA.drop(columns=[col])
print(f"Old data shape: {DATA.shape}")
print(f"New data shape: {CLEANED_DATA.shape}")

Old data shape: (199523, 40)
New data shape: (199523, 509)


This transformation dramatically increased the number of columns, but that's a necessary consequence

### Previous Issues
When I used `LabelEncoder` or `pd.factorize`, the string attributes would successfully be converted to integers. However, for categories like "Tax Filer Status" the tree would output a decision like "FILESTAT <= 2." This has no real meaning -- what does it mean to be "less than" nonfiler? Therefore, I elected to go with the OneHotEncoder.

In [5]:
# This rename is now unneccessary, but it keeps the below cells using X and Y
# I was originally splitting the data into a training/testing set, then I realized there was another file 
X_TRAIN, Y_TRAIN = CLEANED_DATA, RESULTS
print(f"Training_X shape {X_TRAIN.shape}\nTraining_Y Shape {Y_TRAIN.shape}")

Training_X shape (199523, 509)
Training_Y Shape (199523,)


## Train model

This cell is mostly a demonstration showing an example of a height 3 tree, and the illustration that comes with it.  
  
NOTE: graphviz requires some extra binaries, and this cell might not run if you don't have them installed.  
`conda install python-graphviz` will work if you're using conda. In case it doesn't work, I've attached an example image to the notebook

In [6]:
clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=3)
clf = clf.fit(X_TRAIN, Y_TRAIN)

dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=CLEANED_DATA.columns,
                                class_names=CLASS_NAMES,
                                rounded=True, filled=True) 
graph = graphviz.Source(dot_data) 
graph

### Creates a png of the graph. Used to generate picture seen below. _No need to run_

In [24]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=CLEANED_DATA.columns,
                                class_names=CLASS_NAMES,
                                rounded=True, filled=True) 
graph = graphviz.Source(dot_data) 
graph.format = 'png'
graph.render('depth_3_tree')

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.573853 to fit


'depth_10_tree.png'

-------------

![image](depth_3_tree.png)

In the above picture, we can see that the program selects binary attributes with `ATTR <= 0.5`, where True is left and False is right. When selecting continuous attributes, it picks a value to split on.  
We can also see that the graph colors nodes by which class the result is expected to belong to. Orange represents 'below 50000' and blue represents 'above 50000'

## Test models of different depth on training data

In [7]:
def test_model_on_data(clf, testing_data_X, testing_data_Y):
    '''This function computes the accuracy of the model on the given set, returning a decimal representation'''
    total = len(RESULTS)
    wrong = 0
    for expected, actual in zip(clf.predict(testing_data_X), testing_data_Y):
        if expected != actual:
            wrong += 1
    return (total-wrong)/total
    

In [8]:
for i in range(2, 11):
    clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=i)
    clf = clf.fit(X_TRAIN, Y_TRAIN)
    print(f"Max depth {i}: {test_model_on_data(clf, X_TRAIN, Y_TRAIN)}")

Max depth 2: 0.9379419916500854
Max depth 3: 0.941665873107361
Max depth 4: 0.9441267422803387
Max depth 5: 0.9461465595445137
Max depth 6: 0.946958496012991
Max depth 7: 0.9483217473674714
Max depth 8: 0.9507826165404489
Max depth 9: 0.9523613818958215
Max depth 10: 0.9537597169248658


Based on this data, the best value is $k = 10$

# Question 2B

## Load data from Testing Set

In [9]:
TESTING_DATA = pd.read_csv('census-income.test', header=None, names=ATTRIBUTES)
Y_TEST = TESTING_DATA[:]['RESULT']

Y_TEST = Y_TEST.apply(lambda x: 0 if '-' in x else 1)
CLASS_NAMES=['-50000', '50000+']

TESTING_DATA = TESTING_DATA.drop(columns=["MARSUPWT","RESULT"])
ATTRIBUTE_COLUMNS = TESTING_DATA.columns

In [10]:
ohe = OneHotEncoder()
X_TEST = TESTING_DATA.copy()
# These attributes are already integers, so get_dummies will ignore them. We check for them separately
numerical_non_continuous = {'ADTIND', 'ADTOCC','SEOTR','VETYN'}

for t, col in zip(TESTING_DATA.dtypes, TESTING_DATA.columns):
    if t == 'object':
        new_cols = pd.get_dummies(TESTING_DATA[[col]])
        X_TEST = pd.concat([X_TEST, new_cols], axis=1)
        X_TEST = X_TEST.drop(columns=[col])
    if col in numerical_non_continuous:
        new_cols = pd.DataFrame(ohe.fit_transform(TESTING_DATA[[col]]).toarray())
        new_cols.columns = ohe.get_feature_names_out()     
        X_TEST = pd.concat([X_TEST, new_cols], axis=1)
        X_TEST = X_TEST.drop(columns=[col])
print(f"Old data shape: {TESTING_DATA.shape}, {Counter(TESTING_DATA.dtypes)}")
print(f"New data shape: {X_TEST.shape}, {Counter(X_TEST.dtypes)}")

Old data shape: (99762, 40), Counter({dtype('O'): 28, dtype('int64'): 12})
New data shape: (99762, 508), Counter({dtype('uint8'): 395, dtype('float64'): 105, dtype('int64'): 8})


Note that the new testing data shape is 508, meaning that the testing data is missing an example with a certain value for an attribute.  
We will figure out what column that is, then add a 0 column with that attribute as the header. The value should be zero for every instance if we never saw it during the `pd.getdummies` call

In [11]:
# Add in the missing columns
for c in X_TRAIN.columns:
    if c not in X_TEST.columns:
        print(f"Missing: \n\t{c}")
        missing_column = pd.DataFrame([0]*len(X_TEST))
        missing_column.columns = [str(c)]
        X_TEST = pd.concat([X_TEST, missing_column], axis=1)
        
# Rearrange the columns in X_TEST to match the order of X_TRAIN
train_cols = X_TRAIN.columns
X_TEST = X_TEST[train_cols]

print(f"Training shape {X_TRAIN.shape}")
print(f"Testing shape {X_TEST.shape}")

Missing: 
	HHDFMX_ Grandchild <18 ever marr not in subfamily
Training shape (199523, 509)
Testing shape (99762, 509)


### Data is now formatted. Generate Decision Tree and test it against training data and testing data

In [12]:
CHOSEN_K = 10
clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=CHOSEN_K)
clf = clf.fit(X_TRAIN, Y_TRAIN)

In [13]:
print(f"Accuracy on TESTING data: {test_model_on_data(clf, X_TEST, Y_TEST)}")

Accuracy on TESTING data: 0.9751507345017868


# Question 2C

The amount of overfitting can be found by subtracting the error on the training data from the error on the testing data: 
$$ O = e_{test} - e_{train} $$

In [14]:
# The function I wrote returns accuracy, so we do 1-accuracy to get error
train_accuracy = 1 - test_model_on_data(clf, X_TRAIN, Y_TRAIN)
test_accuracy = 1 - test_model_on_data(clf, X_TEST, Y_TEST)
print(f"Error on TRAINING data: {train_accuracy:.5f}")
print(f"Error on TESTING data: {test_accuracy:.5f}")
print(f"Overfitting: {test_accuracy-train_accuracy:.5f}")

Error on TRAINING data: 0.04625
Error on TESTING data: 0.02485
Overfitting: -0.02140
